## Query Processing
---
Separate functions have been created for the 5 types of operations: AND, OR, NOT, AND NOT, OR NOT.
The index and docIDs are loaded from their respective dumps.
The Query is processed from left to right.  

**Input Format**    

Input 1 : Query Sentence  
Input 2 : Operators separted using ", " without any enclosing square brackets  

**Output Format**  

Number of documents matched: _int_  
Number of comparisons required: _int_  
Documents: _List of Document Names_  


In [1]:
# Importing relevent modules for Querying

import re
import string
import codecs
import pickle

from tqdm import tqdm
from pathlib import Path
from substitutions import appos
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from collections import OrderedDict 

In [2]:
# Loading Pickle Dumps

with open("../Dumps/index.pkl","rb") as file:
    postings = pickle.load(file)
    
with open("../Dumps/docID.pkl","rb") as file:
    docID = pickle.load(file)

In [3]:
# Function to clean document text

def clean(text):
    
    # Converting all text to lowercase
    text = text.lower()
        
    # Substituting words with apostrophe with their appropiate phrases
    text = ' '.join([appos[word] if word in appos else word for word in word_tokenize(text)])
    
    # Removing all punctuation and unecessary characters from text
    text = re.sub(r'[^a-z\s]', '', text)
    
    # Removing stopwords from text
    stop_words = set(stopwords.words("english"))
    text = ' '.join([w for w in word_tokenize(text) if not w in stop_words])
    
    # Lemmatizing
    lemmatizer = WordNetLemmatizer()
    text = ' '.join([lemmatizer.lemmatize(w) for w in word_tokenize(text) if not w in stop_words])
    
    return text

In [4]:
# Implementing OR operation

def OR(postings1, postings2):
    
    i = 0
    j = 0
    n = len(postings1)
    m = len(postings2)
    mergedPostings = []
    comparisons = 0
    
    while i < n and j < m:
        comparisons += 1
        
        if postings1[i] < postings2[j]:
            mergedPostings.append(postings1[i])
            i += 1
        elif postings1[i] > postings2[j]:
            mergedPostings.append(postings2[j])
            j += 1
        else:
            mergedPostings.append(postings1[i])
            i += 1
            j += 1
    
    while i < n:
        mergedPostings.append(postings1[i])
        i += 1
        
    while j < m:
        mergedPostings.append(postings2[j])
        j += 1

    return mergedPostings, comparisons

In [5]:
# Implementing AND operation

def AND(postings1, postings2):
    
    i = 0
    j = 0
    n = len(postings1)
    m = len(postings2)
    mergedPostings = []
    comparisons = 0
    
    while i < n and j < m:
        comparisons += 1
        
        if postings1[i] < postings2[j]:
            i += 1
        elif postings1[i] > postings2[j]:
            j += 1
        else:
            mergedPostings.append(postings1[i])
            i += 1
            j += 1
            
    return mergedPostings, comparisons

In [6]:
# Implementing NOT operation

def NOT(postings, totalDocs = 467):
    
    i = 0
    j = 0
    n = len(postings)
    negativePostings = []
    comparisons = 0
    
    while j < totalDocs:
        comparisons += 1
        
        if i<n and postings[i]==j:
            i+=1
        else:
            negativePostings.append(j)
        j+=1
    
    return negativePostings, comparisons

In [7]:
# Implementing AND NOT operation

def ANDNOT(postings1, postings2):
    
    i = 0
    j = 0
    n = len(postings1)
    m = len(postings2)
    mergedPostings = []
    comparisons = 0
    
    while i < n and j < m:
        comparisons += 1
    
        if postings1[i] == postings2[j]:
            i += 1
            j += 1
        elif postings1[i] < postings2[j]:
            mergedPostings.append(postings1[i])
            i += 1
        else:
            j += 1
    
    while i < n:
        mergedPostings.append(postings1[i])
        i += 1
    
    return mergedPostings, comparisons

In [8]:
# Implementing OR NOT operation

def ORNOT(postings1, postings2):
    
    notPostings2, notComparisons = NOT(postings2)
    answer, orComparisons = OR(postings1, notPostings2)
    
    return answer, notComparisons + orComparisons

In [9]:
# Processing Query Left to Right

operations = {'OR': OR,'AND': AND,'OR NOT': ORNOT,'AND NOT': ANDNOT}

def LeftToRightQuery(queryWords, queryOperations):
    
    global postings
    queryPostings = []

    if len(queryWords)==0:
        return [],0
    
    if len(queryOperations)==0:
        if queryWords[0] in postings:
            return postings[queryWords[0]], 0
        else:
            return [], 0
    
    for word in queryWords:
        if word in postings:
            queryPostings.append(postings[word])
        else:
            queryPostings.append([])

    answer = queryPostings[0]
    totalComparisons = 0
    comparisons = 0
    
    for i in range(len(queryOperations)):
        answer, comparisons = operations[queryOperations[i]](answer, queryPostings[i + 1])
        totalComparisons += comparisons
    
    return answer, totalComparisons    

In [10]:
# Handling Input

N = int(input("Enter the number of queries: "))

for n in range(N):
    
    Input_1 = input("Enter query sentence: ").strip()
    queryWords = clean(Input_1).split()
    
    Input_2 = list(map(str,input("Enter query operators: ").split(', ')))
    queryOperation = []
    
    for i in Input_2:
        if i in operations:
            queryOperation.append(i)
    
    if len(queryWords)-len(queryOperation) == 1:    
        answer, comparisons = LeftToRightQuery(queryWords, queryOperation)
    else:
        print("Error")
    
    print("Number of documents matched: ", len(answer))
    print("Number of comparisons required: ", comparisons)
    print("Documents:\n",[docID[i] for i in answer])
    print()

Enter the number of queries: 2
Enter query sentence: lion stood thoughtfully for a moment
Enter query operators: OR, OR, OR
Number of documents matched:  270
Number of comparisons required:  671
Documents:
 ['13chil.txt', '14.lws', '16.lws', '18.lws', '19.lws', '20.lws', '3gables.txt', '3lpigs.txt', '3student.txt', '4moons.txt', '5orange.txt', '6napolen.txt', '7voysinb.txt', 'ab40thv.txt', 'abbey.txt', 'adv_alad.txt', 'advsayed.txt', 'aesop11.txt', 'aesopa10.txt', 'aisle.six', 'aislesix.txt', 'alad10.txt', 'aluminum.hum', 'angelfur.hum', 'angry_ca.txt', 'aquith.txt', 'arcadia.sty', 'archive', 'arctic.txt', 'beast.asc', 'beautbst.txt', 'beggars.txt', 'bestwish', 'beyond.hum', 'bgcspoof.txt', 'bishop00.txt', 'blabnove.hum', 'blabnove.txt', 'blackp.txt', 'blak', 'blasters.fic', 'blh.txt', 'blind.txt', 'blue', 'bluebrd.txt', 'bookem.1', 'bookem2', 'bookem3', 'brain.damage', 'bran', 'breaks1.asc', 'breaks2.asc', 'breaks3.asc', 'bruce-p.txt', 'buggy.txt', 'buldream.txt', 'bulfelis.txt', 'bul

---

In [11]:
# # Processing Query in Efficient Order with Precedence

# def QueryWithPrecedence(queryWords, queryOperation):
    
#     global postings
#     queryPostings = []
#     totalOperations = 0
    
#     for word in queryWords:
#         if word in postings:
#             queryPostings.append(postings[word])
#         else:
#             queryPostings.append([])
    
#     if len(queryOperation) == 1 and queryOperation[0] == "AND NOT":
#         return ANDNOT(queryPostings[0], queryPostings[1])
        
#     for i in range(len(queryOperation)):
    
#         if queryOperation[i] == "OR NOT":
#             queryPostings[i + 1], operations = NOT(queryPostings[i + 1])
#             totalOperations += operations
#             queryOperation[i] = "OR"
        
#         elif queryOperation[i] == "AND NOT":
#             queryPostings[i + 1], operations = NOT(queryPostings[i + 1])
#             totalOperations += operations
#             queryOperation[i] = "AND"
    
#     newPostings = []
#     i = 0
    
#     while i < len(queryWords):
#         currPostings = [queryPostings[i]]
    
#         while i < len(queryOperation) and queryOperation[i] == "AND":
#             currPostings.append(queryPostings[i + 1])
#             i += 1
        
#         currPostings.sort(key = len)
#         combinedPosting = currPostings[0]
        
#         for j in range(1, len(currPostings)):
#             combinedPosting, operations = AND(combinedPosting, currPostings[j])
#             totalOperations += operations
        
#         newPostings.append(combinedPosting)
#         i += 1
    
#     newPostings.sort(key = len)
#     combinedPosting = newPostings[0]
    
#     for i in range(1, len(newPostings)):    
#         combinedPosting, operations = OR(combinedPosting, newPostings[i])
#         totalOperations += operations
    
#     return combinedPosting, totalOperations